In [1]:
!pip install psycopg2

In [15]:
!pip install python-dotenv

In [47]:
import psycopg2
from dotenv import load_dotenv
import os

In [48]:
load_dotenv()  # Load environment variables from .env file

True

In [49]:
# Read connection paramenter from environment variables (BEST PRACTICE!)
host = os.environ.get('DB_HOST')
dbname = os.environ.get('DB_NAME')
user = os.environ.get('DB_USER')
password = os.environ.get('DB_PASS')

In [65]:
## Parameterization is NEEDED!
csv_file_path = 'C:/Users/angel/Desktop/DE PROJECTS/Professional-Portfolio/Building Data Model and Writing ETL Job/Data/airports.csv'
csv_file_path2 = 'C:/Users/angel/Desktop/DE PROJECTS/Professional-Portfolio/Building Data Model and Writing ETL Job/Data/flights.csv'

tbl_airport = 'airport'
tbl_flights = 'flights'

In [45]:
print(f"host={host}") # Testing if env variables were successfully loaded

host=localhost


In [50]:
print(f"host={dbname}") # Testing if env variables were successfully loaded

host=postgres


In [69]:
# Start connection to Postgres Database 
# using WITH (context manager) to ensure the connection is properly closed (BEST PRACTICE!), 
try:
    with psycopg2.connect(f"host={host} dbname='raw_inventory' user={user} password={password}") as conn:
        # Create a cursor object
        with conn.cursor() as cursor:
            # Create a table if not exists
            create_airport_table = "CREATE TABLE IF NOT EXISTS airport (airport_id INT, city VARCHAR, state VARCHAR, name VARCHAR)"
            create_flights_table = "CREATE TABLE IF NOT EXISTS flights (DayofMonth INT, DayOfWeek INT, Carrier VARCHAR, OriginAirportID INT, DestAirportID INT, DepDelay INT, ArrDelay INT)"
            cursor.execute(create_airport_table)
            cursor.execute(create_flights_table)

            # Read CSV file and load data into tables
            with open(csv_file_path, 'r') as file:
                next(file) #skip first row
                cursor.execute(f"TRUNCATE TABLE {tbl_airport}")
                cursor.copy_from(file, tbl_airport, sep=',' )

            with open(csv_file_path2, 'r') as file:
                next(file) #skip first row
                cursor.execute(f"TRUNCATE TABLE {tbl_flights}")
                cursor.copy_from(file, tbl_flights, sep=',' )

    conn.commit()
except psycopg2.OperationalError as e:
    print("Could not start a connection to the database")
    print(e)
except psycopg2.DatabaseError as e:
    print("An error occur while working with the database")
    print(e)

host=localhost
